In [1]:
import pandas as pd
import numpy as np

In [2]:
unit = {"A":5, "B":10, "C":5, "D":15}
maturity = {"A":5, "B":7, "C":10, "D":15}
coupon = {"A":0.05, "B":0.03, "C":0.01, "D":0.07}
ytm = {"A":0.025, "B":0.03, "C":0.035, "D":0.035}
bond = pd.DataFrame({"unit":unit,'maturity':maturity, 'coupon':coupon, "ytm":ytm})

In [3]:
def pv(bond, k):
    pv = []
    for i in range(len(bond)):
        c = bond.iloc[i][2]*100/k
        r = bond.iloc[i][3]/k
        m = bond.iloc[i][1]*k
        pv.append(c/r*(1-1/(1+r)**m)+100/(1+r)**m)
    bond['pv'] = pv

In [4]:
pv(bond, 2)

In [5]:
def MD(bond,k):
    md_ = []
    for i in range(len(bond)):
        m = int(bond.iloc[i][1])*k
        c = bond.iloc[i][2]*100/k
        r = bond.iloc[i][3]/k
        pv = bond.iloc[i][4]
        md = 0
        t = 1/k
        for j in range(1,m):
            md += c/(1+r)**j/pv*t
            
            t += 1/k 
        md += (c+100)/(1+r)**m/pv*t
        md_.append(md/(1+r))
    bond['MD'] = md_

In [6]:
MD(bond,2)

In [7]:
bond

,unit,maturity,coupon,ytm,pv,MD
A,5,5,0.05,0.025,111.681907,4.462799
B,10,7,0.03,0.030,100.000000,6.271691
C,5,10,0.01,0.035,79.058898,9.303421
D,15,15,0.07,0.035,140.575236,10.261723


In [8]:
def Cx(bond,k):
    cx_ = []
    for i in range(len(bond)):
        m = int(bond.iloc[i][1])*k
        c = bond.iloc[i][2]*100/k
        r = bond.iloc[i][3]/k
        pv = bond.iloc[i][4]
        cx = 0
        for j in range(1, m):
            cx += c/(1+r)**j*(j*j+j)
        cx += (c+100)/(1+r)**m*(m*m+m)
        cx_.append(cx/(k*k*pv*(1+r)**2))
    bond['Cx'] = cx_

In [9]:
Cx(bond,2)

In [10]:
bond

,unit,maturity,coupon,ytm,pv,MD,Cx
A,5,5,0.05,0.025,111.681907,4.462799,23.395571
B,10,7,0.03,0.030,100.000000,6.271691,44.848681
C,5,10,0.01,0.035,79.058898,9.303421,94.293080
D,15,15,0.07,0.035,140.575236,10.261723,135.347689


In [11]:
def portPV(bond):
    v = 0
    for i in range(len(bond)):
        v += bond.iloc[i][4] * bond.iloc[i][0]
    
    return v

In [12]:
portPV(bond)

4062.332574023996

In [13]:
def weight(bond):
    v = portPV(bond)
    
    w = []
    for i in range(len(bond)):
        w.append(bond.iloc[i][4] * bond.iloc[i][0]/v)
    bond['weight'] = w

In [14]:
weight(bond)

In [15]:
bond

,unit,maturity,coupon,ytm,pv,MD,Cx,weight
A,5,5,0.05,0.025,111.681907,4.462799,23.395571,0.137460
B,10,7,0.03,0.030,100.000000,6.271691,44.848681,0.246164
C,5,10,0.01,0.035,79.058898,9.303421,94.293080,0.097307
D,15,15,0.07,0.035,140.575236,10.261723,135.347689,0.519068


In [16]:
def portMD(bond):
    p = 0
    for i in range(len(bond)):
        p += bond.iloc[i][5]*bond.iloc[i][7]
    return p

In [17]:
print(portMD(bond))

8.389149202447218


In [18]:
def portCx(bond):
    p = 0
    for i in range(len(bond)):
        p += bond.iloc[i][6]*bond.iloc[i][7]
    return p

In [19]:
print(portCx(bond))

93.68620612260227


In [20]:
p = pd.Series(data = {"pv": portPV(bond), "MD":portMD(bond), "Cx":portCx(bond)},name='Portfolio')
bond = bond.append(p,ignore_index=False)

In [21]:
bond

,unit,maturity,coupon,ytm,pv,MD,Cx,weight
A,5.0,5.0,0.05,0.025,111.681907,4.462799,23.395571,0.137460
B,10.0,7.0,0.03,0.030,100.000000,6.271691,44.848681,0.246164
C,5.0,10.0,0.01,0.035,79.058898,9.303421,94.293080,0.097307
D,15.0,15.0,0.07,0.035,140.575236,10.261723,135.347689,0.519068
Portfolio,NaN,NaN,NaN,NaN,4062.332574,8.389149,93.686206,NaN


In [22]:
unit = {"E":1, "F":1, "G":1}
maturity = {"E":5, "F":10000, "G":12}
coupon = {"E":0, "F":0.05, "G":0.03}
ytm = {"E":0.03, "F":0.06, "G":0.035}
bond2 = pd.DataFrame({"unit":unit,'maturity':maturity, 'coupon':coupon, "ytm":ytm})

In [23]:
pv(bond2,2)
MD(bond2,2)
Cx(bond2,2)

In [24]:
bond2

,unit,maturity,coupon,ytm,pv,MD,Cx
E,1,5,0.00,0.030,86.166723,4.926108,26.693198
F,1,10000,0.05,0.060,83.333333,16.666667,555.555556
G,1,12,0.03,0.035,95.134829,9.934654,114.905579


In [25]:
with pd.ExcelWriter('hw1.xlsx') as writer:
    bond.to_excel(writer,sheet_name='ABCD')
    bond2.to_excel(writer,sheet_name='EFG')

In [26]:
print(bond)
print(bond2)

           unit  maturity  coupon    ytm           pv         MD          Cx  \
A           5.0       5.0    0.05  0.025   111.681907   4.462799   23.395571   
B          10.0       7.0    0.03  0.030   100.000000   6.271691   44.848681   
C           5.0      10.0    0.01  0.035    79.058898   9.303421   94.293080   
D          15.0      15.0    0.07  0.035   140.575236  10.261723  135.347689   
Portfolio   NaN       NaN     NaN    NaN  4062.332574   8.389149   93.686206   

             weight  
A          0.137460  
B          0.246164  
C          0.097307  
D          0.519068  
Portfolio       NaN  
   unit  maturity  coupon    ytm         pv         MD          Cx
E     1         5    0.00  0.030  86.166723   4.926108   26.693198
F     1     10000    0.05  0.060  83.333333  16.666667  555.555556
G     1        12    0.03  0.035  95.134829   9.934654  114.905579


In [27]:
import numpy as np

P = np.array([[8.72, 4.85, 16.67], [101.37, 28.28, 555.56], [1, 1, 1]])
# P_inv = np.linalg.inv(P)
O = np.array([[-8.72], [1111.12], [1]])

print(P)
# X = A * P_inv
# X = P_inv * A
X = np.linalg.solve(P, O)
print(X)

[[  8.72   4.85  16.67]
 [101.37  28.28 555.56]
 [  1.     1.     1.  ]]
[[-16.95862133]
 [ 13.55423347]
 [  4.40438786]]


In [28]:
P = np.array([[8.72, 4.85, 16.67], [101.37, 8.28, 555.56], [1, 1, 1]])
# P_inv = np.linalg.inv(P)
O = np.array([[17.44], [55.556], [1]])

print(P)
# X = A * P_inv
# X = P_inv * A
X = np.linalg.solve(P, O)
print(X)

[[  8.72   4.85  16.67]
 [101.37   8.28 555.56]
 [  1.     1.     1.  ]]
[[ 6.22164214]
 [-4.24975085]
 [-0.97189129]]


In [29]:
P = np.array([[8.72, 4.85, 16.67], [101.37, 8.28, 555.56], [1, 1, 1]])
# P_inv = np.linalg.inv(P)
O = np.array([[-8.72], [1111.12], [1]])

print(P)
# X = A * P_inv
# X = P_inv * A
X = np.linalg.solve(P, O)
print(X)

[[  8.72   4.85  16.67]
 [101.37   8.28 555.56]
 [  1.     1.     1.  ]]
[[-20.10726912]
 [ 15.67197881]
 [  5.43529031]]


In [30]:
print('done')

done
